# LABORATORIO 2: Optimización de Redes

Jairo Garavito Correa - 202111499

In [3]:
from pyomo.environ import *

from pyomo.opt import SolverFactory

import matplotlib.pyplot as plt

import pandas as pd

## Problem 1: transport networks

Sets:

* Origin cities: S = {Bogotá, Medellín}
* destination cities: D = {Cali, Barranquilla, Pasto, Tunja, Chía, Manizales}

Parameters:

* Offer of the origin cities:
$$O_s = \forall s \in S$$

* Demand of the destination cities:
$$P_d = \forall d \in D$$

* Cost to send products from a supplier city to a customer city:
$$E_{s,d} = \forall {s \in S, d \in D}$$

Decision variables:

* Tons of products to be shipped from each city of origin to each destination city:
$$X_{s,d} = \forall {s \in S, d \in D}$$

Objective function:

* Minimize transport cost:
$$\text{Minimize} \sum_{s \in S} \sum_{d \in D} X_{s,d} \times E_{s,d}$$

Constraints:

* Demand of destination cities:

$$\sum_{s \in S}  X_{s,d} = P_d \space \forall d \in D $$

* Offer of the origin cities:

$$\sum_{d \in D}  X_{s,d} \leq O_s \space \forall s \in S $$


In [4]:
model = ConcreteModel()

# Conjuntos
model.S = ['Bogotá', 'Medellín']  # Ciudades de origen
model.D = ['Cali', 'Barranquilla', 'Pasto', 'Tunja', 'Chía', 'Manizales']  # Ciudades de destino

# Parámetros
# Oferta en las ciudades de origen
model.O = {'Bogotá': 550, 'Medellín': 700}

# Demanda en las ciudades de destino
model.P = {'Cali': 125, 'Barranquilla': 175, 'Pasto': 225, 'Tunja': 250, 'Chía': 225, 'Manizales': 200}

# Costos de transporte de las ciudades de origen a las de destino
model.E = {
    ('Bogotá', 'Cali'): 100000000,
    ('Bogotá', 'Barranquilla'): 2.5,
    ('Bogotá', 'Pasto'): 1.6,
    ('Bogotá', 'Tunja'): 1.4,
    ('Bogotá', 'Chía'): 0.8,
    ('Bogotá', 'Manizales'): 1.4,
    ('Medellín', 'Cali'): 2.5,
    ('Medellín', 'Barranquilla'): 100000000,
    ('Medellín', 'Pasto'): 2.0,
    ('Medellín', 'Tunja'): 1.0,
    ('Medellín', 'Chía'): 1.0,
    ('Medellín', 'Manizales'): 0.8
}

# Variables de decisión
model.X = Var(model.S, model.D, domain=NonNegativeReals)

# Función objetivo: minimizar el costo de transporte
def objective_rule(model):
    return summation(model.E, model.X)
model.OBJ = Objective(rule=objective_rule, sense=1)  # sense=1 para minimizar

# Restricciones

# Restricción de demanda en las ciudades de destino
def demand_rule(model, d):
    return sum(model.X[s, d] for s in model.S) >= model.P[d]
model.DemandConstraint = Constraint(model.D, rule=demand_rule)

# Restricción de oferta en las ciudades de origen
def offer_rule(model, s):
    return sum(model.X[s, d] for d in model.D) <= model.O[s]
model.OfferConstraint = Constraint(model.S, rule=offer_rule)

# Resolver el modelo
solver = SolverFactory('glpk')
solver.solve(model)

model.display()

Model unknown

  Variables:
    X : Size=12, Index={Bogotá, Medellín}*{Cali, Barranquilla, Pasto, Tunja, Chía, Manizales}
        Key                          : Lower : Value : Upper : Fixed : Stale : Domain
          ('Bogotá', 'Barranquilla') :     0 : 175.0 :  None : False : False : NonNegativeReals
                  ('Bogotá', 'Cali') :     0 :   0.0 :  None : False : False : NonNegativeReals
                  ('Bogotá', 'Chía') :     0 : 150.0 :  None : False : False : NonNegativeReals
             ('Bogotá', 'Manizales') :     0 :   0.0 :  None : False : False : NonNegativeReals
                 ('Bogotá', 'Pasto') :     0 : 225.0 :  None : False : False : NonNegativeReals
                 ('Bogotá', 'Tunja') :     0 :   0.0 :  None : False : False : NonNegativeReals
        ('Medellín', 'Barranquilla') :     0 :   0.0 :  None : False : False : NonNegativeReals
                ('Medellín', 'Cali') :     0 : 125.0 :  None : False : False : NonNegativeReals
                ('Medell

## Problem 2: Optimal Routes for teams

Sets:

* Locations: N = {0,1,2,3,4,5}

Parameters:

* cost of transport
$$c_{ij} \quad \forall i \in N, \forall j \in N$$

Decision variables:

$$x_{ij} \quad \forall i \in N, \forall j \in N$$

$$u_{i} \quad \forall i \in N$$

Objective function:

$$
\begin{aligned}
\text{Minimize:} \quad & \sum_{i \in N} \sum_{j \in N} c_{ij} \cdot x_{ij} \\
\text{Subject to:} \quad & \sum_{j \in N, j \neq i} x_{ij} = 1 \quad \forall i \in N \quad \text{(Each city is left once)} \\
                         & \sum_{i \in N, i \neq j} x_{ij} = 1 \quad \forall j \in N \quad \text{(Each city is visited once)} \\
                         & u_i - u_j + (n-1) \cdot x_{ij} \leq n-1 \quad \forall i,j \in N, i \neq j \quad \text{(MTZ subtour elimination)} \\
                         & u_i \in \mathbb{R}, \quad u_1 = 1 \\
                         & x_{ij} \in \{0, 1\} \quad \forall i,j \in N
\end{aligned}
$$

In [22]:

#Modelo
model = ConcreteModel()

#Conjuntos
localidades = {0,1,2,3,4,5}

#Parametros
n = len(localidades)

df = pd.read_csv('./data/proof_case.csv')
df.columns = [int(col.strip(" ")) for col in df.columns]
df = df.apply(pd.to_numeric)
for i in range(len(df)):
    df.iloc[i, i] = 999

costos = {}
for origen in localidades:
    for destino in localidades:
        costos[(origen, destino)] = int(df[origen][destino])

print(costos)
         
#Variable de decision
model.x = Var(localidades, localidades, domain=Binary)

#Funcion Objetivo
model.obj = Objective(
    expr = sum(sum(model.x[(i,j)]*costos[i,j] for j in localidades) for i in localidades), 
    sense = 1
)

#Restricciones
def destiny_once(model, i):
    return sum(model.x[(i,j)] for j in localidades if j!=i) == 1
model.destiny_once = Constraint(localidades, rule=destiny_once)

def origin_once(model, j):
    return sum(model.x[(i,j)] for i in localidades if i!=j) == 1
model.origin_once = Constraint(localidades, rule=origin_once)


# Variable de orden de los nodos para evitar subtours
model.u = Var(localidades, domain=NonNegativeReals)

# Restricción de eliminación de subtours
def subtour_elimination(model, i, j):
    if i != 0 and j != 0 and i != j:
        return model.u[i] - model.u[j] + n * model.x[(i, j)] <= n - 1
    else:
        return Constraint.Skip
model.subtour_elimination = Constraint(localidades, localidades, rule=subtour_elimination)

#Modelo
SolverFactory('glpk').solve(model)
model.display()

{(0, 0): 999, (0, 1): 1, (0, 2): 1, (0, 3): 2, (0, 4): 2, (0, 5): 1, (1, 0): 1, (1, 1): 999, (1, 2): 3, (1, 3): 2, (1, 4): 3, (1, 5): 2, (2, 0): 1, (2, 1): 3, (2, 2): 999, (2, 3): 3, (2, 4): 3, (2, 5): 1, (3, 0): 2, (3, 1): 2, (3, 2): 3, (3, 3): 999, (3, 4): 1, (3, 5): 3, (4, 0): 2, (4, 1): 3, (4, 2): 2, (4, 3): 1, (4, 4): 999, (4, 5): 3, (5, 0): 1, (5, 1): 3, (5, 2): 3, (5, 3): 2, (5, 4): 3, (5, 5): 999}


Model unknown

  Variables:
    x : Size=36, Index={0, 1, 2, 3, 4, 5}*{0, 1, 2, 3, 4, 5}
        Key    : Lower : Value : Upper : Fixed : Stale : Domain
        (0, 0) :     0 :   0.0 :     1 : False : False : Binary
        (0, 1) :     0 :   1.0 :     1 : False : False : Binary
        (0, 2) :     0 :   0.0 :     1 : False : False : Binary
        (0, 3) :     0 :   0.0 :     1 : False : False : Binary
        (0, 4) :     0 :   0.0 :     1 : False : False : Binary
        (0, 5) :     0 :   0.0 :     1 : False : False : Binary
        (1, 0) :     0 :   0.0 :     1 : False : False : Binary
        (1, 1) :     0 :   0.0 :     1 : False : False : Binary
        (1, 2) :     0 :   0.0 :     1 : False : False : Binary
        (1, 3) :     0 :   1.0 :     1 : False : False : Binary
        (1, 4) :     0 :   0.0 :     1 : False : False : Binary
        (1, 5) :     0 :   0.0 :     1 : False : False : Binary
        (2, 0) :     0 :   0.0 :     1 : False : False : Binary
        (2, 1) 

## Problem 3: Optimization of Sensor Placement in Smart Cities

Sets:

* Locations: L = {L1, L2, L3, L4, L5, L6, L7, L8, L9, L10, L11, L12}

* Sensors: S = {Enviromental Sensor, Traffic Sensor, Public Safety Sensor}

Parameters:

* Energy consumption cost of sensor s.
$$E_{s} \space \forall s \in S$$

* Cost of installation at the location l.
$$I_{l} \space \forall l \in L$$

* Cost of communication of a sensor s in a location l.
$$C_{s,l} \space \forall s \in S, l \in L$$

* Binary parameter indicating whether sensor s can cover location l directly.
$$k_{s,l} \space \forall s \in S, l \in L$$

* Binary parameter indicating whether location l1 is adjacent to l2.
$$A_{l_1,l_2} \space \forall l_{1} \in L, l_{2} \in L$$

Decision variables:
* Binary decision variable indicating whether sensor s is conected at location l.
$$X_{s,l} \in \{0,1\}$$

* Binary decision variable indicating whether sensor s is selected.
$$Y_{s} \in \{0,1\}$$

Objective function:

$$\text{Minimize} \sum_{s \in S} \left( E_s \times Y_s \right) + \sum_{s \in S} \sum_{l \in L} \left( C_{s,l} \times X_{s,l} \right) + \sum_{s \in S} \sum_{l \in L} \left( I_{l} \times X_{s,l} \right)$$

Constraints:

$$ \sum_{l_1 \in L} A_{l_1, l_2} \times X_{s,l_2} \geq k_{s,l_2} \space \forall l_2 \in L, s \in S$$

$$ X_{s,l} \leq Y_{s} \space \forall s \in S, l \in L $$

In [43]:
#Modelo
model = ConcreteModel()

#Conjuntos
localidades_df = pd.read_csv('./data/locations.csv')
localidades = RangeSet(len(localidades_df.columns))

sensores_df = pd.read_csv('./data/sensors.csv')
sensores = RangeSet(len(sensores_df.columns))

print(localidades)
print(sensores)

#Parametros
sensor_coverage_df = pd.read_csv('./data/sensor_coverage.csv')
sensor_coverage_df = sensor_coverage_df.drop(['Location'], axis=1)
sensor_coverage_df.columns = [sensor for sensor in sensores]
sensor_coverage = {}
for localidad in localidades:
    for sensor in sensores:
        sensor_coverage[(localidad, sensor)] = sensor_coverage_df[sensor][localidad-1]

print(sensor_coverage)

energy_consumption_df = pd.read_csv('./data/energy_consumption.csv')
energy_consumption_df = energy_consumption_df.drop(['SensorType'], axis=1)
energy_consumption = {}
for sensor in sensores:
    energy_consumption[sensor] = energy_consumption_df['EnergyConsumption'][sensor-1]   

communication_costs_df = pd.read_csv('./data/communication_costs.csv')
communication_costs = {}
for index, row in communication_costs_df.iterrows():
    localidad = int(row['Location'].strip("L"))
    sensor = int(row['SensorType'].strip("S"))
    costo = row['CommunicationCost']
    communication_costs[(localidad,sensor)] = costo 

installation_costs_df = pd.read_csv('./data/installation_costs.csv')
installation_costs_df = installation_costs_df.drop(['Location'], axis=1)
installation_costs = {}
for localidad in localidades:
    installation_costs[localidad] = installation_costs_df['InstallationCost'][localidad-1]    
    
zone_coverage_df = pd.read_csv('./data/zone_coverage.csv')
zone_coverage_df = zone_coverage_df.drop(['Location'], axis=1)
zone_coverage_df.columns = [location for location in localidades]

zone_coverage = {}
for localidad in localidades:
    for localidad2 in localidades:
        zone_coverage[(localidad, localidad2)] = zone_coverage_df[localidad][localidad2-1]

print(zone_coverage)

model.X = Var(sensores, localidades, within=Binary)  # Si se instala un sensor en una localidad
model.Y = Var(sensores, within=Binary)           # Si se selecciona un tipo de sensor

def objective_function(model):
    return (
        sum(energy_consumption[s] * model.Y[s] for s in sensores) +
        sum(communication_costs[l,s] * model.X[s, l] for l in localidades for s in sensores) +
        sum(installation_costs[l] * model.X[s, l] for l in localidades for s in sensores)
    )
    
model.obj = Objective(rule=objective_function, sense=minimize)

# 1. Restricción de cobertura de zonas
def coverage_constraint(model, l2, s):
    return sum(zone_coverage[l1, l2] * model.X[s,l1] for l1 in localidades) >= sensor_coverage[l2, s]

model.coverage_constraint = Constraint(localidades, sensores, rule=coverage_constraint)

# 2. Restricción de que si un sensor está instalado en una ubicación, debe ser seleccionado
def sensor_selection_constraint(model, l, s):
    return model.X[s, l] <= model.Y[s]

model.sensor_selection_constraint = Constraint(localidades, sensores, rule=sensor_selection_constraint)

# Resolver el modelo
solver = SolverFactory('glpk')
result = solver.solve(model)

# Mostrar resultados
model.display()

[1:12]
[1:3]
{(1, 1): 1, (1, 2): 0, (1, 3): 1, (2, 1): 1, (2, 2): 0, (2, 3): 1, (3, 1): 1, (3, 2): 0, (3, 3): 1, (4, 1): 1, (4, 2): 1, (4, 3): 0, (5, 1): 1, (5, 2): 1, (5, 3): 1, (6, 1): 1, (6, 2): 1, (6, 3): 1, (7, 1): 1, (7, 2): 0, (7, 3): 1, (8, 1): 1, (8, 2): 1, (8, 3): 1, (9, 1): 1, (9, 2): 1, (9, 3): 1, (10, 1): 1, (10, 2): 1, (10, 3): 0, (11, 1): 1, (11, 2): 1, (11, 3): 1, (12, 1): 1, (12, 2): 0, (12, 3): 1}
{(1, 1): 1, (1, 2): 1, (1, 3): 1, (1, 4): 0, (1, 5): 1, (1, 6): 0, (1, 7): 0, (1, 8): 0, (1, 9): 0, (1, 10): 0, (1, 11): 0, (1, 12): 0, (2, 1): 1, (2, 2): 1, (2, 3): 0, (2, 4): 0, (2, 5): 1, (2, 6): 0, (2, 7): 0, (2, 8): 0, (2, 9): 0, (2, 10): 0, (2, 11): 0, (2, 12): 0, (3, 1): 1, (3, 2): 0, (3, 3): 1, (3, 4): 1, (3, 5): 1, (3, 6): 1, (3, 7): 1, (3, 8): 1, (3, 9): 0, (3, 10): 0, (3, 11): 0, (3, 12): 0, (4, 1): 0, (4, 2): 0, (4, 3): 1, (4, 4): 1, (4, 5): 1, (4, 6): 1, (4, 7): 0, (4, 8): 0, (4, 9): 0, (4, 10): 0, (4, 11): 1, (4, 12): 0, (5, 1): 1, (5, 2): 1, (5, 3): 1, (5, 4):